In [ ]:
# https://github.com/jukyellow/algoTrade/blob/master/ch04/4.6%20%EB%A7%88%EB%B2%95%EA%B3%B5%EC%8B%9D%20%EA%B5%AC%ED%98%84.ipynb

In [4]:
pip install -U finance-datareader

In [42]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

In [43]:
krx_df = fdr.StockListing('KRX')
krx_df.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [7]:
#PER: 자본수익률, ROA: 이익수익률
def sort_value(someValue, asc=True, standard=0): # 특정 지표를 오름차순(내림차순=False)으로 순위를 매김
    
    someValue_mask = someValue.mask(someValue<standard, np.nan)
    someValue_rank = someValue_mask.rank(axis=1, ascending=asc, na_option="bottom")
    
    return someValue_rank

def MagicFormula(PER, ROA): #마법공식
    
    PER_rank = sort_value(PER, asc=True)
    ROA_rank = sort_value(ROA, asc=False)
    
    result_rank = PER_rank + ROA_rank
    
    result = result_rank.rank(axis=1, ascending=True)
    
    result = result.where(result <= 10, 0)
    result = result.mask(result > 0, 1)
    
    return result

In [8]:
krx_df['Symbol'][0]

'060310'

In [44]:
df = pd.read_csv('PER_ROA.csv',engine='python', encoding='cp949')
print(df.size)
df.head()

15300


,종목명,현재가,전일비,등락률,액면가,거래량,시가총액,영업이익,PER,ROA
0,삼성전자,43950,100,-0.23%,100,4637815,2623719,588867.0,7.30,13.83
1,SK하이닉스,74400,600,0.81%,5000,1673056,541634,208438.0,3.49,28.49
2,삼성전자우,36350,350,-0.95%,100,553133,299119,NaN,6.03,NaN
3,현대차,126000,2000,1.61%,5000,252713,269222,24222.0,23.54,0.92
4,NAVER,148500,500,0.34%,100,367244,244748,9425.0,37.72,7.02


In [45]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
print(df.size)
df.head()

7580


,종목명,현재가,전일비,등락률,액면가,거래량,시가총액,영업이익,PER,ROA
0,삼성전자,43950,100,-0.23%,100,4637815,2623719,588867.0,7.30,13.83
1,SK하이닉스,74400,600,0.81%,5000,1673056,541634,208438.0,3.49,28.49
3,현대차,126000,2000,1.61%,5000,252713,269222,24222.0,23.54,0.92
4,NAVER,148500,500,0.34%,100,367244,244748,9425.0,37.72,7.02
5,LG화학,329500,9500,2.97%,5000,176590,232602,22461.0,17.52,5.63


In [19]:
# with open 구문으로 csv 읽기
# import csv
# line_list = []
# with open('PER_ROA.csv') as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',') #  ',' 구분자로 파일을 읽는다.
#     for row in csv_reader:  
#         if '' in row:     # 한줄씩 순회하는데 NaN이 포함된 Row는 건너뛴다.
#             pass
#         else : 
#             line_list.append(row) # 리스트에 추가.

# df = pd.DataFrame(data=line_list[1:],columns=line_list[0])
# print(df.size)
# df.head()

UnicodeDecodeError: ignored

In [22]:
def sort_value2( s_value, asc = True, standard = 0): 
    '''
    description
        특정 지표값을 정렬한다.
        
    parameters
        s_value : pandas Series
            정렬할 데이터를 받는다.
            
        asc : bool
            True : 오름차순
            False : 내림차순
            
        standard : int
            조건에 맞는 값을 True로 대체하기 위한 기준값 
    returns 
        s_value_mask_rank : pandas Series
            정렬된 순위
    ''' 
    
    s_value_mask = s_value.mask(s_value < standard, np.nan)  #지표별 기준값 미만은 필터링 한다.
    s_value_mask_rank = s_value_mask.rank( ascending=asc, na_option="bottom") # 필터링된 종목에서 순위 선정
    
    return s_value_mask_rank

In [46]:
per = pd.to_numeric(df['PER'])
roa = pd.to_numeric(df['ROA'])
print(type(per))
print(per.size)
print(per.head())
print()
print(roa.head())
print()

<class 'pandas.core.series.Series'>
758
0     7.30
1     3.49
3    23.54
4    37.72
5    17.52
Name: PER, dtype: float64

0    13.83
1    28.49
3     0.92
4     7.02
5     5.63
Name: ROA, dtype: float64



In [47]:
# per = df['PER']
per_rank = sort_value2(per, asc=True, standard=0 ) # PER 지표값을 기준으로 순위 정렬 및 0 미만 값 제거
roa_rank = sort_value2(roa, asc=False, standard=0 )# ROA 지표값을 기준으로 순위 정렬 및 0 미만 값 제거

#소팅이 제대로 된거 맞아??
print(per_rank.head())
print(roa_rank.head())
print(per_rank.size)

0    139.0
1     22.0
3    424.0
4    487.0
5    367.0
Name: PER, dtype: float64
0     18.0
1      5.0
3    478.5
4    103.0
5    149.0
Name: ROA, dtype: float64
758


In [48]:
result_rank = per_rank + roa_rank   # PER 순위 ROA 순위 합산
result_rank = sort_value2(result_rank, asc=True)  # 합산 순위 정렬
result_rank = result_rank.where(result_rank <= 10, 0)  # 합산 순위 필터링
result_rank = result_rank.mask(result_rank > 0, 1)  # 순위 제거

print(result_rank.sum())
print(result_rank.size)
result_rank.head(20)

10.0
758


0     0.0
1     1.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
dtype: float64

In [49]:
mf_df = df.loc[result_rank > 0,['종목명','시가총액']].copy() # 선택된 종목 데이터프레임 복사
mf_df.head()

,종목명,시가총액
1,SK하이닉스,541634
107,효성,18353
199,대한유화,7508
200,HDC,7468
371,한일홀딩스,2904


In [50]:
mf_stock_list = df.loc[result_rank > 0, '종목명'].values # 선택된 종목명 추출
print(mf_stock_list)

['SK하이닉스' '효성' '대한유화' 'HDC' '한일홀딩스' '신대양제지' '세아제강지주' 'SIMPAC' '케이씨'
 '에쓰씨엔지니어링']


In [51]:
mf_df['종목코드'] = ''
for stock in mf_stock_list :
    # 데이터프레임에서 조건에 맞는 데이터만 불러온다.
    mf_df.loc[mf_df['종목명'] == stock,'종목코드'] = krx_df[krx_df['Name'] == stock]['Symbol'].values
mf_df.head()

,종목명,시가총액,종목코드
1,SK하이닉스,541634,000660
107,효성,18353,004800
199,대한유화,7508,006650
200,HDC,7468,012630
371,한일홀딩스,2904,003300


In [54]:
# 수익률 구하는 방식 1

mf_df['2019_수익률'] = ''
for x in mf_df['종목코드'].values : 
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    cum_ret = df.loc[df.index[-1], 'Close'] / df.loc[df.index[0],'Close'] -1  # 2019년도 누적 수익률 계산
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

mf_df

,종목명,시가총액,종목코드,2019_수익률
1,SK하이닉스,541634,000660,0.552805
107,효성,18353,004800,0.649635
199,대한유화,7508,006650,-0.20339
200,HDC,7468,012630,-0.332326
371,한일홀딩스,2904,003300,-0.197441
383,신대양제지,2756,016590,0.0347826
450,세아제강지주,2044,003030,-0.0083682
474,SIMPAC,1835,009160,0.206967
494,케이씨,1694,029460,0.594017
938,에쓰씨엔지니어링,354,023960,-0.0620915


In [55]:
#수익률 방식2

mf_df_rtn = pd.DataFrame()
for x in mf_df['종목코드'].values : 
#     print(x ,', ' , mf_df.loc[mf_df['종목코드'] == x, '종목명' ].values[0])
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    df['daily_rtn'] = df['Close'].pct_change(periods=1)
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod()
    cum_ret = df.loc[df.index[-1],'cum_rtn']
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

mf_df

,종목명,시가총액,종목코드,2019_수익률
1,SK하이닉스,541634,000660,1.55281
107,효성,18353,004800,1.64964
199,대한유화,7508,006650,0.79661
200,HDC,7468,012630,0.667674
371,한일홀딩스,2904,003300,0.802559
383,신대양제지,2756,016590,1.03478
450,세아제강지주,2044,003030,0.991632
474,SIMPAC,1835,009160,1.20697
494,케이씨,1694,029460,1.59402
938,에쓰씨엔지니어링,354,023960,0.937908


In [56]:
# 수익률 방식3

for ind,val in enumerate(mf_df['종목코드'].values) : 
    # 가독성을 위해 종목명 추출.
    code_name = mf_df.loc[mf_df['종목코드'] == val,'종목명'].values[0]
    print(val, code_name)
    df = fdr.DataReader(val, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    if ind == 0 :
        mf_df_rtn = pd.DataFrame(index=df.index) # 첫 번째 종목코드 인덱스 활용한 데이터프레임 생성
    df['daily_rtn'] = df['Close'].pct_change(periods=1) # period 기간 차이만큼 변동율 계산
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod() # 누적 곱 계산
    tmp = df.loc[:,['cum_rtn']].rename(columns={'cum_rtn':code_name}) # 가독성을 위한 컬럼명 변경
    mf_df_rtn = mf_df_rtn.join(tmp,how='left') # 새로 계산된 누적 수익률 추가.
    df = None # 데이터 프레임 초기화

mf_df_rtn.tail()

000660 SK하이닉스
004800 효성
006650 대한유화
012630 HDC
003300 한일홀딩스
016590 신대양제지
003030 세아제강지주
009160 SIMPAC
029460 케이씨
023960 에쓰씨엔지니어링


,SK하이닉스,효성,대한유화,HDC,한일홀딩스,신대양제지,세아제강지주,SIMPAC,케이씨,에쓰씨엔지니어링
Date,,,,,,,,,,
2019-12-23,1.561056,1.722628,0.820339,0.655589,0.814442,1.041739,0.974895,1.192623,1.337607,0.947712
2019-12-24,1.547855,1.747654,0.820339,0.655589,0.804388,1.026087,0.990586,1.178279,1.354701,0.908497
2019-12-26,1.564356,1.758081,0.823729,0.667674,0.820841,1.045217,1.003138,1.192623,1.410256,0.928105
2019-12-27,1.584158,1.660063,0.803390,0.664653,0.800731,1.041739,0.991632,1.200820,1.534188,0.911765
2019-12-30,1.552805,1.649635,0.796610,0.667674,0.802559,1.034783,0.991632,1.206967,1.594017,0.937908
